In [1]:
import openpathsampling as paths
import numpy as np

Load first frame from test pdb.

In [2]:
template = paths.tools.snapshot_from_pdb('../data/Alanine_solvated.pdb')

Create a simple CV without an underlying function.

In [4]:
cv0 = paths.CollectiveVariable('func0')

Create a complicated function.

In [5]:
center = 1
def dist(snapshot, center, np):
    return np.sum(snapshot.coordinates._value[0]) - center

Create collective variable from this function. Note that you have to specify `center` and `np` to make this work

In [6]:
cv1 = paths.CV_Function('func1', dist, center=center, np=np)
cv2 = paths.CV_Function('func2', dist, center=center, np=np, cv_wrap_numpy_array=True)
cv3 = paths.CV_Function('func3', dist, center=center, np=np, cv_wrap_numpy_array=True, cv_time_reversible=True)

Create storage to test save and load.

In [7]:
storage = paths.Storage('can_be_deleted.nc', mode='w', template=template)

Save CV

In [8]:
storage.save([cv0, cv1, cv2, cv3])

[(store.cvs[CollectiveVariable], 6, 0),
 (store.cvs[CollectiveVariable], 6, 1),
 (store.cvs[CollectiveVariable], 6, 2),
 (store.cvs[CollectiveVariable], 6, 3)]

Test function for reversed template.

In [9]:
print dist(template.reversed, center, np)

-1.08320000023


Create another CV. This time using the `from_template` function.

In [9]:
print cv0([template, template])
print cv1([template, template])

[-1.0832000002264977, -1.0832000002264977]
[-1.0832 -1.0832]


In [10]:
print type(cv0([template, template]))
print type(cv0([template, template])[0])
print type(cv1([template, template]))
print type(cv1([template, template])[0])

<type 'list'>
<type 'numpy.float64'>
<type 'numpy.ndarray'>
<type 'numpy.float64'>


In [11]:
#! skip
print storage.cvs.variables['json'][0]
print storage.cvs.variables['json'][1]
print storage.cvs.variables['json'][2]

{"_cls": "CV_Function", "_dict": {"name": "func1", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTMtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB", "_module_vars": [], "_global_vars": []}, "cv_store_cache": true, "cv_time_reversible": false, "cv_wrap_numpy_array": false, "cv_requires_lists": false, "cv_scalarize_numpy_singletons": false, "**kwargs": {"np": {"_import": "numpy"}, "center": 1}}}
{"_cls": "CV_Function", "_dict": {"name": "func2", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzGwAAAHwCAGoAAHwAAGoBAGoCAGQBABmDAQB8AQAYUygCAAAATmkAAAAAKAMAAAB0AwAAAHN1bXQLAAAAY29vcmRpbmF0ZXN0BgAAAF92YWx1ZSgDAAAAdAgAAABzbmFwc2hvdHQGAAAAY2VudGVydAIAAABucCgAAAAAKAAAAABzHgAAADxpcHl0aG9uLWlucHV0LTMtNzMyMzhiNjY5OTg3PnQEAAAAZGlzdAIAAABzAgAAAAAB", "_module_vars": [], "_global_vars": []}, "cv_store_cache": true, "cv_t

In [12]:
cv0j = storage.cvs.vars['json'][0]
cv1j = storage.cvs.vars['json'][1]
cv2j = storage.cvs.vars['json'][2]

In [13]:
res = cv0j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv1j([template, template, template])
assert(res[0] == res[1] == res[2])
res = cv2j([template, template, template])
assert(res[0] == res[1] == res[2])

In [14]:
print storage.variables['snapshots_cv_0_values_fw']
print storage.variables['snapshots_cv_1_values_fw']
print storage.variables['snapshots_cv_2_values_fw']

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_0_values_fw(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_1_values_fw(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used

<type 'netCDF4._netCDF4.Variable'>
float64 snapshots_cv_2_values_fw(snapshots)
    var_type: numpy.float64
    maskable: True
unlimited dimensions: snapshots
current shape = (1,)
filling on, default _FillValue of 9.96920996839e+36 used



In [15]:
print storage.cvs.cache_var(0)[:]
print storage.cvs.cache_var(1)[:]
print storage.cvs.cache_var(2)[:]

[None]
[None]
[None]


Sync again.

In [16]:
cv0.sync()
cv1.sync()
cv2.sync()

Note that the third CV, which had `time_reversible=True` will automatically compute the values for the reversed snapshot, too. So after syncing we see two equal values.

In [22]:
print storage.cvs.cache_var(0)[:]
print storage.cvs.cache_bw(0)[:]
print storage.cvs.cache_var(1)[:]
print storage.cvs.cache_bw(1)[:]
print storage.cvs.cache_var(2)[:]
print storage.cvs.cache_bw(2)[:]

[-1.0832000002264977]
[None]
[-1.0832000002264977]
[None]
[-1.0832000002264977]


KeyError: 'snapshots_cv_2_values_bw'

Since our function returns a numpy.float64 scalar we get these back (hopefully)

In [18]:
print type(storage.vars['snapshots_cv_0_values_fw'][0])
print type(storage.vars['snapshots_cv_1_values_fw'][0])
print type(storage.vars['snapshots_cv_2_values_fw'][0])

<type 'numpy.float64'>
<type 'numpy.float64'>
<type 'numpy.float64'>


In [19]:
print cv0._store_dict.cache
print cv1._store_dict.cache
print cv2._store_dict.cache

LRUCache(1/0 of 1000000/0)
LRUCache(1/0 of 1000000/0)
LRUCache(2/0 of 1000000/0)
